# 分類とQ&A評価でのlogprobsの使用

このノートブックでは、Chat Completions APIの`logprobs`パラメータの使用方法を説明します。`logprobs`が有効になっている場合、APIは各出力トークンの対数確率と、各トークン位置での最も可能性の高いトークンの限定数とその対数確率を返します。関連するリクエストパラメータは以下の通りです：
* `logprobs`: 出力トークンの対数確率を返すかどうか。trueの場合、メッセージのコンテンツで返される各出力トークンの対数確率を返します。
* `top_logprobs`: 各トークン位置で返す最も可能性の高いトークンの数を指定する0から5の間の整数で、それぞれに関連する対数確率が付きます。このパラメータを使用する場合は、`logprobs`をtrueに設定する必要があります。

出力トークンの対数確率は、コンテキストが与えられた場合の各トークンがシーケンス内で発生する可能性を示します。簡単に言うと、logprobは`log(p)`で、`p` = コンテキスト内の前のトークンに基づいて特定の位置でトークンが発生する確率です。`logprobs`に関する重要なポイント：
* より高い対数確率は、そのコンテキストでのトークンの可能性がより高いことを示します。これにより、ユーザーはモデルの出力に対する信頼度を測定したり、モデルが検討した代替的な応答を探索したりできます。
* Logprobは任意の負の数または`0.0`になります。`0.0`は100%の確率に対応します。
* Logprobsにより、個々のトークンのlogprobsの合計としてシーケンスの結合確率を計算できます。これはモデル出力のスコアリングとランキングに有用です。もう一つの一般的なアプローチは、最良の生成を選択するために文のトークンあたりの平均logprobを取ることです。
* 異なる候補トークンに割り当てられた`logprobs`を調べることで、モデルがどのオプションを妥当または不妥当と考えたかを理解できます。

`logprobs`には幅広い使用例がありますが、このノートブックでは以下の用途に焦点を当てます：

1. 分類タスク

* 大規模言語モデルは多くの分類タスクに優れていますが、モデルの出力に対する信頼度を正確に測定することは困難な場合があります。`logprobs`は各クラス予測に関連する確率を提供し、ユーザーが独自の分類または信頼度の閾値を設定できるようにします。

2. 検索（Q&A）評価

* `logprobs`は検索アプリケーションでの自己評価を支援できます。Q&Aの例では、モデルは作為的な`has_sufficient_context_for_answer`ブール値を出力し、これは答えが検索されたコンテンツに含まれているかどうかの信頼度スコアとして機能します。この種の評価は検索ベースの幻覚を減らし、精度を向上させることができます。

3. オートコンプリート
*  `logprobs`は、ユーザーが入力している際に単語を提案する方法を決定するのに役立ちます。

4. トークンハイライトとバイト出力
*  ユーザーは`logprobs`を有効にすることで付属する組み込みトークン化を使用して、トークンハイライターを簡単に作成できます。さらに、bytesパラメータには各出力文字のASCIIエンコーディングが含まれており、これは絵文字や特殊文字を再現するのに特に有用です。

5. パープレキシティの計算
* `logprobs`は、結果に対するモデルの全体的な信頼度を評価し、異なるプロンプトからの結果の信頼度を比較するのに役立ちます。

## 0. インポートとユーティリティ

In [ ]:
from openai import OpenAI
from math import exp
import numpy as np
from IPython.display import display, HTML
import os

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

In [31]:
def get_completion(
    messages: list[dict[str, str]],
    model: str = "gpt-4",
    max_tokens=500,
    temperature=0,
    stop=None,
    seed=123,
    tools=None,
    logprobs=None,  # whether to return log probabilities of the output tokens or not. If true, returns the log probabilities of each output token returned in the content of message..
    top_logprobs=None,
) -> str:
    params = {
        "model": model,
        "messages": messages,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "stop": stop,
        "seed": seed,
        "logprobs": logprobs,
        "top_logprobs": top_logprobs,
    }
    if tools:
        params["tools"] = tools

    completion = client.chat.completions.create(**params)
    return completion

## 1. 分類タスクにおける信頼度評価のための`logprobs`の使用

ニュース記事を事前定義されたカテゴリのセットに分類するシステムを作成したいとしましょう。`logprobs`なしでも、Chat Completionsを使用してこれを行うことができますが、モデルがその分類をどの程度の確信度で行ったかを評価することははるかに困難です。

現在、`logprobs`を有効にすることで、モデルがその予測にどの程度確信を持っているかを正確に確認できるようになりました。これは、正確で信頼できる分類器を作成する上で非常に重要です。例えば、選択されたカテゴリの対数確率が高い場合、これはモデルがその分類に非常に確信を持っていることを示唆します。低い場合は、モデルの確信度が低いことを示唆します。これは、モデルの分類が期待したものではない場合や、モデルの出力を人間がレビューまたは検証する必要がある場合に特に有用です。

モデルに4つのカテゴリ（**テクノロジー、政治、スポーツ、芸術**）を提示するプロンプトから始めます。モデルは、記事の見出しのみに基づいて、これらのカテゴリに記事を分類するタスクを実行します。

In [32]:
CLASSIFICATION_PROMPT = """You will be given a headline of a news article.
Classify the article into one of the following categories: Technology, Politics, Sports, and Art.
Return only the name of the category, and nothing else.
MAKE SURE your output is one of the four categories stated.
Article headline: {headline}"""


3つのサンプル見出しを見て、まず`logprobs`を使わない標準的なChat Completionsの出力から始めましょう。

In [33]:
headlines = [
    "Tech Giant Unveils Latest Smartphone Model with Advanced Photo-Editing Features.",
    "Local Mayor Launches Initiative to Enhance Urban Public Transport.",
    "Tennis Champion Showcases Hidden Talents in Symphony Orchestra Debut",
]


In [69]:
for headline in headlines:
    print(f"\nHeadline: {headline}")
    API_RESPONSE = get_completion(
        [{"role": "user", "content": CLASSIFICATION_PROMPT.format(headline=headline)}],
        model="gpt-4o",
    )
    print(f"Category: {API_RESPONSE.choices[0].message.content}\n")


Headline: Tech Giant Unveils Latest Smartphone Model with Advanced Photo-Editing Features.
Category: Technology


Headline: Local Mayor Launches Initiative to Enhance Urban Public Transport.
Category: Politics


Headline: Tennis Champion Showcases Hidden Talents in Symphony Orchestra Debut
Category: Art



ここでは、各見出しに対して選択されたカテゴリを確認できます。しかし、モデルの予測に対する信頼度は見えません。同じプロンプトを再実行してみましょう。今度は`logprobs`を有効にし、`top_logprobs`を2に設定します（これにより、各トークンに対して最も可能性の高い2つの出力トークンが表示されます）。さらに、各出力トークンの線形確率も出力できます。これにより、対数確率をより解釈しやすい0-100%のスケールに変換できます。

In [57]:
for headline in headlines:
    print(f"\nHeadline: {headline}")
    API_RESPONSE = get_completion(
        [{"role": "user", "content": CLASSIFICATION_PROMPT.format(headline=headline)}],
        model="gpt-4o-mini",
        logprobs=True,
        top_logprobs=2,
    )
    top_two_logprobs = API_RESPONSE.choices[0].logprobs.content[0].top_logprobs
    html_content = ""
    for i, logprob in enumerate(top_two_logprobs, start=1):
        html_content += (
            f"<span style='color: cyan'>Output token {i}:</span> {logprob.token}, "
            f"<span style='color: darkorange'>logprobs:</span> {logprob.logprob}, "
            f"<span style='color: magenta'>linear probability:</span> {np.round(np.exp(logprob.logprob)*100,2)}%<br>"
        )
    display(HTML(html_content))
    print("\n")


Headline: Tech Giant Unveils Latest Smartphone Model with Advanced Photo-Editing Features.





Headline: Local Mayor Launches Initiative to Enhance Urban Public Transport.





Headline: Tennis Champion Showcases Hidden Talents in Symphony Orchestra Debut


最初の2つの見出しから予想される通り、gpt-4o-miniはその分類に100%の確信を持っています。これは、コンテンツがそれぞれ明確にテクノロジーと政治に焦点を当てているためです。しかし、3つ目の見出しはスポーツとアート関連のテーマの両方を組み合わせているため、信頼度がわずかに低い97%となっていますが、それでも分類に対して強い確信を示しています。

`logprobs`は分類タスクにおいて非常に有用です。これにより、信頼度の閾値を設定したり、選択された出力の対数確率が十分に高くない場合に複数の潜在的なトークンを出力したりすることができます。例えば、記事にタグ付けするレコメンデーションエンジンを作成する際、特定の閾値を超える見出しを自動的に分類し、確信度の低いものは手動レビューに送ることができます。

## 2. 幻覚を減らすための検索信頼度スコアリング

幻覚を減らし、RAGベースのQ&Aシステムのパフォーマンスを向上させるために、`logprobs`を使用してモデルが検索にどの程度確信を持っているかを評価することができます。

以下のテキストを日本語に翻訳します：

Q&A用にRAGを使用した検索システムを構築したが、質問に対する幻覚的な回答に悩まされているとしましょう。*注意:* この例では、ハードコードされた記事を使用しますが、Q&AでRAGを使用するチュートリアルについては、クックブックの他のエントリを参照してください。

In [36]:
# Article retrieved
ada_lovelace_article = """Augusta Ada King, Countess of Lovelace (née Byron; 10 December 1815 – 27 November 1852) was an English mathematician and writer, chiefly known for her work on Charles Babbage's proposed mechanical general-purpose computer, the Analytical Engine. She was the first to recognise that the machine had applications beyond pure calculation.
Ada Byron was the only legitimate child of poet Lord Byron and reformer Lady Byron. All Lovelace's half-siblings, Lord Byron's other children, were born out of wedlock to other women. Byron separated from his wife a month after Ada was born and left England forever. He died in Greece when Ada was eight. Her mother was anxious about her upbringing and promoted Ada's interest in mathematics and logic in an effort to prevent her from developing her father's perceived insanity. Despite this, Ada remained interested in him, naming her two sons Byron and Gordon. Upon her death, she was buried next to him at her request. Although often ill in her childhood, Ada pursued her studies assiduously. She married William King in 1835. King was made Earl of Lovelace in 1838, Ada thereby becoming Countess of Lovelace.
Her educational and social exploits brought her into contact with scientists such as Andrew Crosse, Charles Babbage, Sir David Brewster, Charles Wheatstone, Michael Faraday, and the author Charles Dickens, contacts which she used to further her education. Ada described her approach as "poetical science" and herself as an "Analyst (& Metaphysician)".
When she was eighteen, her mathematical talents led her to a long working relationship and friendship with fellow British mathematician Charles Babbage, who is known as "the father of computers". She was in particular interested in Babbage's work on the Analytical Engine. Lovelace first met him in June 1833, through their mutual friend, and her private tutor, Mary Somerville.
Between 1842 and 1843, Ada translated an article by the military engineer Luigi Menabrea (later Prime Minister of Italy) about the Analytical Engine, supplementing it with an elaborate set of seven notes, simply called "Notes".
Lovelace's notes are important in the early history of computers, especially since the seventh one contained what many consider to be the first computer program—that is, an algorithm designed to be carried out by a machine. Other historians reject this perspective and point out that Babbage's personal notes from the years 1836/1837 contain the first programs for the engine. She also developed a vision of the capability of computers to go beyond mere calculating or number-crunching, while many others, including Babbage himself, focused only on those capabilities. Her mindset of "poetical science" led her to ask questions about the Analytical Engine (as shown in her notes) examining how individuals and society relate to technology as a collaborative tool.
"""

# Questions that can be easily answered given the article
easy_questions = [
    "What nationality was Ada Lovelace?",
    "What was an important finding from Lovelace's seventh note?",
]

# Questions that are not fully covered in the article
medium_questions = [
    "Did Lovelace collaborate with Charles Dickens",
    "What concepts did Lovelace build with Charles Babbage",
]

今度は、モデルに質問に回答してもらうだけでなく、その回答を評価してもらうことができます。具体的には、モデルにブール値の`has_sufficient_context_for_answer`を出力するよう求めます。そして、`logprobs`を評価することで、提供されたコンテキストに答えが含まれていたことについて、モデルがどれほど確信を持っているかを確認できます。

In [61]:
PROMPT = """You retrieved this article: {article}. The question is: {question}.
Before even answering the question, consider whether you have sufficient information in the article to answer the question fully.
Your output should JUST be the boolean true or false, of if you have sufficient information in the article to answer the question.
Respond with just one word, the boolean true or false. You must output the word 'True', or the word 'False', nothing else.
"""


In [65]:
html_output = ""
html_output += "Questions clearly answered in article"

for question in easy_questions:
    API_RESPONSE = get_completion(
        [
            {
                "role": "user",
                "content": PROMPT.format(
                    article=ada_lovelace_article, question=question
                ),
            }
        ],
        model="gpt-4o-mini",
        logprobs=True,
    )
    html_output += f'<p style="color:green">Question: {question}</p>'
    for logprob in API_RESPONSE.choices[0].logprobs.content:
        html_output += f'<p style="color:cyan">has_sufficient_context_for_answer: {logprob.token}, <span style="color:darkorange">logprobs: {logprob.logprob}, <span style="color:magenta">linear probability: {np.round(np.exp(logprob.logprob)*100,2)}%</span></p>'

html_output += "Questions only partially covered in the article"

for question in medium_questions:
    API_RESPONSE = get_completion(
        [
            {
                "role": "user",
                "content": PROMPT.format(
                    article=ada_lovelace_article, question=question
                ),
            }
        ],
        model="gpt-4o",
        logprobs=True,
        top_logprobs=3,
    )
    html_output += f'<p style="color:green">Question: {question}</p>'
    for logprob in API_RESPONSE.choices[0].logprobs.content:
        html_output += f'<p style="color:cyan">has_sufficient_context_for_answer: {logprob.token}, <span style="color:darkorange">logprobs: {logprob.logprob}, <span style="color:magenta">linear probability: {np.round(np.exp(logprob.logprob)*100,2)}%</span></p>'

display(HTML(html_output))

最初の2つの質問について、我々のモデルは記事が提示された質問に答えるのに十分なコンテキストを持っていることを（ほぼ）100%の信頼度で断言しています。<br><br>
一方、記事内でより明確に答えられていない、より難しい質問については、モデルは十分なコンテキストを持っているという信頼度が低くなります。これは、取得したコンテンツが十分であることを確保するための優れたガードレールです。<br><br>
この自己評価は、`sufficient_context_for_answer`の対数確率が特定の閾値を下回った場合に回答を制限したり、ユーザーに再プロンプトを促したりすることで、ハルシネーションを減らすのに役立ちます。このような手法は、Q&AにおけるRAGのハルシネーションやエラーを大幅に削減することが示されています（[例](https://jfan001.medium.com/how-we-cut-the-rate-of-gpt-hallucinations-from-20-to-less-than-2-f3bfcc10e4ec)）

## 3. オートコンプリート

`logprobs`のもう一つの使用例は、オートコンプリートシステムです。オートコンプリートシステム全体をエンドツーエンドで作成することなく、ユーザーが入力している際に単語を提案する方法を決定するために`logprobs`がどのように役立つかを実演してみましょう。

まず、サンプル文を考えてみましょう：`"My least favorite TV show is Breaking Bad."`この文を入力している際に、次の単語やトークンを動的に推薦したいとします。ただし、モデルが次の単語について*かなり確信を持っている*場合のみです。これを実演するために、文を順次的な構成要素に分解してみましょう。

In [39]:
sentence_list = [
    "My",
    "My least",
    "My least favorite",
    "My least favorite TV",
    "My least favorite TV show",
    "My least favorite TV show is",
    "My least favorite TV show is Breaking Bad",
]

これで、`gpt-4o-mini`に対して、モデルに与えられたコンテキストに基づいてオートコンプリートエンジンとして動作するよう指示できます。`logprobs`を有効にすることで、モデルがその予測にどの程度確信を持っているかを確認できます。

In [ ]:
high_prob_completions = {}
low_prob_completions = {}
html_output = ""

for sentence in sentence_list:
    PROMPT = """Complete this sentence. You are acting as auto-complete. Simply complete the sentence to the best of your ability, make sure it is just ONE sentence: {sentence}"""
    API_RESPONSE = get_completion(
        [{"role": "user", "content": PROMPT.format(sentence=sentence)}],
        model="gpt-4o-mini",
        logprobs=True,
        top_logprobs=3,
    )
    html_output += f'<p>Sentence: {sentence}</p>'
    first_token = True
    for token in API_RESPONSE.choices[0].logprobs.content[0].top_logprobs:
        html_output += f'<p style="color:cyan">Predicted next token: {token.token}, <span style="color:darkorange">logprobs: {token.logprob}, <span style="color:magenta">linear probability: {np.round(np.exp(token.logprob)*100,2)}%</span></p>'
        if first_token:
            if np.exp(token.logprob) > 0.95:
                high_prob_completions[sentence] = token.token
            if np.exp(token.logprob) < 0.60:
                low_prob_completions[sentence] = token.token
        first_token = False
    html_output += "<br>"

display(HTML(html_output))

高信頼度の自動補完を見てみましょう：

In [48]:
high_prob_completions


{'My least favorite TV show is Breaking Bad': 'because'}

これらは妥当に見えます！これらの提案に自信を持つことができます。「My least favorite TV」と書いた後に「show」と書きたくなる可能性は非常に高いでしょう！では次に、モデルがあまり確信を持てなかった自動補完の提案を見てみましょう：

In [49]:
low_prob_completions


{'My least favorite TV': 'show', 'My least favorite TV show': 'is'}

これらも論理的です。「my least favorite」という接頭辞だけでは、ユーザーが何を言おうとしているのかかなり不明確ですし、著者のお気に入りのテレビ番組が何なのかは本当に誰にも分かりません。<br><br>
そこで、`gpt-4o-mini`を使用して、`logprobs`で動的自動補完エンジンの基盤を作ることができます！

## 4. ハイライターとbytesパラメータ

`logprobs`を使用したシンプルなトークンハイライターの作成と、bytesパラメータの使用について簡単に説明しましょう。まず、各トークンをカウントしてハイライトする関数を作成できます。これは対数確率を使用しませんが、`logprobs`を有効にすることで利用できる組み込みのトークン化機能を使用します。

In [66]:
PROMPT = """What's the longest word in the English language?"""

API_RESPONSE = get_completion(
    [{"role": "user", "content": PROMPT}], model="gpt-4o", logprobs=True, top_logprobs=5
)


def highlight_text(api_response):
    colors = [
        "#FF00FF",  # Magenta
        "#008000",  # Green
        "#FF8C00",  # Dark Orange
        "#FF0000",  # Red
        "#0000FF",  # Blue
    ]
    tokens = api_response.choices[0].logprobs.content

    color_idx = 0  # Initialize color index
    html_output = ""  # Initialize HTML output
    for t in tokens:
        token_str = bytes(t.bytes).decode("utf-8")  # Decode bytes to string

        # Add colored token to HTML output
        html_output += f"<span style='color: {colors[color_idx]}'>{token_str}</span>"

        # Move to the next color
        color_idx = (color_idx + 1) % len(colors)
    display(HTML(html_output))  # Display HTML output
    print(f"Total number of tokens: {len(tokens)}")

In [67]:
highlight_text(API_RESPONSE)


Total number of tokens: 95


次に、bytesパラメータを使用して文を再構築してみましょう。`logprobs`を有効にすると、各トークンとトークン文字列のASCII（10進数のutf-8）値の両方が提供されます。これらのASCII値は、絵文字や特殊文字を含むトークンや、それらで構成されるトークンを処理する際に役立ちます。

In [68]:
PROMPT = """Output the blue heart emoji and its name."""
API_RESPONSE = get_completion(
    [{"role": "user", "content": PROMPT}], model="gpt-4o", logprobs=True
)

aggregated_bytes = []
joint_logprob = 0.0

# Iterate over tokens, aggregate bytes and calculate joint logprob
for token in API_RESPONSE.choices[0].logprobs.content:
    print("Token:", token.token)
    print("Log prob:", token.logprob)
    print("Linear prob:", np.round(exp(token.logprob) * 100, 2), "%")
    print("Bytes:", token.bytes, "\n")
    aggregated_bytes += token.bytes
    joint_logprob += token.logprob

# Decode the aggregated bytes to text
aggregated_text = bytes(aggregated_bytes).decode("utf-8")

# Assert that the decoded text is the same as the message content
assert API_RESPONSE.choices[0].message.content == aggregated_text

# Print the results
print("Bytes array:", aggregated_bytes)
print(f"Decoded bytes: {aggregated_text}")
print("Joint prob:", np.round(exp(joint_logprob) * 100, 2), "%")

Token: Here
Log prob: -0.054242473
Linear prob: 94.72 %
Bytes: [72, 101, 114, 101] 

Token:  is
Log prob: -0.0044352207
Linear prob: 99.56 %
Bytes: [32, 105, 115] 

Token:  the
Log prob: -2.1008714e-06
Linear prob: 100.0 %
Bytes: [32, 116, 104, 101] 

Token:  blue
Log prob: -0.0013290489
Linear prob: 99.87 %
Bytes: [32, 98, 108, 117, 101] 

Token:  heart
Log prob: 0.0
Linear prob: 100.0 %
Bytes: [32, 104, 101, 97, 114, 116] 

Token:  emoji
Log prob: 0.0
Linear prob: 100.0 %
Bytes: [32, 101, 109, 111, 106, 105] 

Token:  and
Log prob: -0.038287632
Linear prob: 96.24 %
Bytes: [32, 97, 110, 100] 

Token:  its
Log prob: 0.0
Linear prob: 100.0 %
Bytes: [32, 105, 116, 115] 

Token:  name
Log prob: -1.569009e-05
Linear prob: 100.0 %
Bytes: [32, 110, 97, 109, 101] 

Token: :


Log prob: -0.11313002
Linear prob: 89.3 %
Bytes: [58, 10, 10] 

Token: \xf0\x9f\x92
Log prob: -0.09048584
Linear prob: 91.35 %
Bytes: [240, 159, 146] 

Token: \x99
Log prob: 0.0
Linear prob: 100.0 %
Bytes: [153] 

Token:

ここでは、最初のトークンが `\xf0\x9f\x92'` であることがわかりますが、そのASCII値を取得してbytes配列に追加することができます。その後、この配列を完全な文に簡単にデコードし、デコードされたbytesが完了メッセージと同じであることをassert文で検証できます！

さらに、完了全体の結合確率を取得することもできます。これは各トークンの対数確率の指数積です。これにより、与えられたプロンプトに対してこの特定の完了がどの程度`可能性が高い`かがわかります。私たちのプロンプトは非常に指示的（特定の絵文字とその名前を求めている）であるため、この出力の結合確率は高くなります！しかし、ランダムな出力を求めた場合、結合確率ははるかに低くなります。これは、プロンプトエンジニアリング中の開発者にとっても良い戦術となり得ます。

## 5. パープレキシティの計算

モデルの結果に対する信頼度を評価する際、不確実性の指標であるパープレキシティを計算することが有用です。パープレキシティは、logprobsの平均値の負の値を指数化することで計算できます。一般的に、パープレキシティが高いほど不確実な結果を示し、パープレキシティが低いほど信頼度の高い結果を示します。そのため、パープレキシティは個々のモデル実行結果を評価するためだけでなく、複数のモデル実行結果間の相対的な信頼度を比較するためにも使用できます。高い信頼度が結果の正確性を保証するわけではありませんが、他の評価指標と組み合わせることで、プロンプトの動作をより深く理解するための有用なシグナルとなります。

例えば、`gpt-4o-mini`を使用して人工知能についてより詳しく学びたいとしましょう。最近の歴史について質問することもできれば、未来について質問することもできます：

In [ ]:
prompts = [
    "In a short sentence, has artifical intelligence grown in the last decade?",
    "In a short sentence, what are your thoughts on the future of artificial intelligence?",
]

for prompt in prompts:
    API_RESPONSE = get_completion(
        [{"role": "user", "content": prompt}],
        model="gpt-4o-mini",
        logprobs=True,
    )

    logprobs = [token.logprob for token in API_RESPONSE.choices[0].logprobs.content]
    response_text = API_RESPONSE.choices[0].message.content
    response_text_tokens = [token.token for token in API_RESPONSE.choices[0].logprobs.content]
    max_starter_length = max(len(s) for s in ["Prompt:", "Response:", "Tokens:", "Logprobs:", "Perplexity:"])
    max_token_length = max(len(s) for s in response_text_tokens)
    

    formatted_response_tokens = [s.rjust(max_token_length) for s in response_text_tokens]
    formatted_lps = [f"{lp:.2f}".rjust(max_token_length) for lp in logprobs]

    perplexity_score = np.exp(-np.mean(logprobs))
    print("Prompt:".ljust(max_starter_length), prompt)
    print("Response:".ljust(max_starter_length), response_text, "\n")
    print("Tokens:".ljust(max_starter_length), " ".join(formatted_response_tokens))
    print("Logprobs:".ljust(max_starter_length), " ".join(formatted_lps))
    print("Perplexity:".ljust(max_starter_length), perplexity_score, "\n")

Prompt:     In a short sentence, has artifical intelligence grown in the last decade?
Response:   Yes, artificial intelligence has grown significantly in the last decade, advancing in capabilities and applications across various fields. 

Tokens:                Yes              ,     artificial   intelligence            has          grown  significantly             in            the           last         decade              ,      advancing             in   capabilities            and   applications         across        various         fields              .
Logprobs:            -0.00           0.00          -0.00           0.00          -0.00          -0.73          -0.00          -0.01          -0.02          -0.00           0.00          -0.02          -0.66          -0.03          -0.62          -0.47          -0.02          -0.39          -0.01          -0.20          -0.00
Perplexity: 1.1644170003987546 

Prompt:     In a short sentence, what are your thoughts on the future of a

この例では、`gpt-4o-mini`は最近の歴史に関するより決定論的な質問に対してより低いperplexityスコアを返し、近い将来に関するより推測的な評価に対してはより高いperplexityスコアを返しました。繰り返しになりますが、これらの違いは正確性を保証するものではありませんが、モデルの結果の解釈と今後の活用方法を示す手がかりとなります。

## 6. 結論

素晴らしい！`logprobs`パラメータを使用して、より堅牢な分類器を構築し、Q&Aシステムの検索を評価し、トークンの各「バイト」をエンコードおよびデコードすることができました！`logprobs`は完了出力に有用な情報とシグナルを追加し、開発者がこれをアプリケーションの改善にどのように組み込むかを見るのが楽しみです。

## 7. 可能な拡張機能

`logprobs`には、このクックブックでカバーされていない他の多くの使用例があります。`logprobs`は以下の用途に使用できます：
  - モデレーション
  - キーワード選択
  - プロンプトの改善と出力の解釈可能性向上
  - トークンヒーリング
  - その他多数！